In [138]:
# info conent reomve unesccary
# binning changes eg installment low in dist high in count
# removed distplot for numeric 
# installment and loan amount
# removed diff and put heat map
# add invalid uvivariate also
# analysis add of segment uni
#how verified is issue

In [139]:
# class imbalance and binning
# drop current loans
# impute only metric say?
# remove outliers? outlier function
# some description in univariate
# tell range of each univariate
# see final count of univariate comment

In [140]:
#chnage dataset
#remove warnings
# \n comments
# convert to markdown type
# assumption : employ length 10> is 10 and <1 is 1
#     heatmap
# ateime convert object
# label and title.

Importing Libraries

In [247]:
#Import pandas lib for data manipulation
import pandas as pd
#Import numpy lib scientific calculation
import numpy as np
#Import seaborn and matplot lib for visulaization of plots
import seaborn as sns
import matplotlib.pyplot as plt

# DATA UNDERSTANDING

Load data and check shape to get number of rows and columns

In [248]:
#load dataset
df = pd.read_csv("loan.csv")
df.shape

So the dataset has 39717 rows and 111 columns.
Let see some samples

In [249]:
df.head(3)

Checking datatype of the variables

In [250]:
df.info()

So there are 74 columns of float type, 13 of int type and 24 of object.
View object type columns to ensure that they are not numeric variables actaully with wrong datatype.

Displaying all variables of datatype object.

In [251]:
df.select_dtypes([object]).columns

The following variables should be actually be numerical:
i)Term
ii)int_rate: interest rate
iii)emp_length:employment length
iv)zip_code
v)revol_util: revoving line utilization rate
So converting them to numeric data type.

Lets see how term varaible looks like.

In [252]:
df.term

Now lets see count of each category in term.

In [253]:
df.term.value_counts()

Changing term variable to interger type.

In [254]:
#changing term column to numerical
df.term = df.term.apply(lambda x:int(x[:3]))
df.term

Lets see how interest rates look like.

In [255]:
df.int_rate

We filter out the interest rate by slicing.

In [256]:
df.int_rate = df.int_rate.apply(lambda x:float(x[:-1]))
df.int_rate

Lets see how the employment length variable looks like.

In [257]:
df.emp_length

Lets see categories in employment length.

In [258]:
df.emp_length.unique()

We will convert to integer type.
For easier analysis,we assume  employment length > 10 years will be taken as 10 years while
less than one year will be treated as one year length.

In [259]:
#Function to convert employment length to integer type. 
def emp_length_fix_datatype(x):
    try:
        # All lengths ablove 10 will be treated as 10
        if x[2] == "+":
            return 10
        #All lengths less than 1 will be treated as 1
        elif x[2] == "1":
            return 1
        else:
            return int(x[0])
    except:
        return x
df.emp_length = df.emp_length.apply(lambda x:emp_length_fix_datatype(x))
df.emp_length.unique()

Lets see how employment length categories look like after conversion.

In [260]:
df.emp_length.value_counts()

Lets see how zip codes look like.

In [261]:
df.zip_code.value_counts()

Since last three digits are hidden, they can be removed and convert to integer type.

In [262]:
df.zip_code = df.zip_code.apply(lambda x:int(x[:3]))
df.zip_code.value_counts()

Lets see how revol_util variable looks like.

In [263]:
df.revol_util.value_counts()

We change it to float type by slicing out the numeric part.

In [264]:
def change_revol_util_to_float(x):
    try:
        return float(x[:-1])
    except:
        return x

df.revol_util = df.revol_util.apply(lambda x: change_revol_util_to_float(x))
df.revol_util

Checking target variable.
Now loan_status is our target variable and it is categorical target,
Lets see the classes in target.

In [265]:
df.loan_status.value_counts()

# DATA CLEANING

Lets first check the missing values.


Checking for missing values percentages of all variables.

In [266]:
df.isna().sum()*100/len(df)

So there are some columns with very high percentage of missing values( some even missing all values)
Dropping columns with more than 80% missing values

In [267]:
#drop missing values >0.8
df = df.loc[:, df.isna().sum() < 0.8*len(df)]


Lets see new shape after dropping columns of high percentage missing values

In [270]:
df.shape

Lets see how many unique values are there in each column.


In [271]:
df.nunique().sort_values()

So there are 7 columns with only one unique value.
These columns can be dropped and useful in analysis as they will be same value irrespective of
whether the loan defaults or not.

Lets recheck using value counts for these columns.

In [272]:
df.columns

In [273]:
df.pymnt_plan.value_counts()

In [274]:
df.initial_list_status.value_counts()

In [275]:
df.application_type.value_counts()

In [276]:
df.collections_12_mths_ex_med.value_counts()

In [277]:
df.chargeoff_within_12_mths.value_counts()

In [278]:
df.tax_liens.value_counts()

In [279]:
df.policy_code.value_counts()

So these columns can be dropped as they wont be useful in prediction.

In [280]:
list_same_value = ["pymnt_plan","initial_list_status","application_type","collections_12_mths_ex_med","chargeoff_within_12_mths","tax_liens","policy_code"]
df = df.drop(list_same_value,axis =1)

Now there are some columns that dont have any useful information for analysis of loan default.
Some of the columns like url, desc, id,member_id dont influence the default rate.
So these can be removed as well.

In [282]:
list_no_influence = ["url", "desc", "id","member_id"]
df = df.drop(list_no_influence,axis =1)

Now 'loan_amnt','funded_amnt','funded_amnt_inv' are usually correlated. 
Lets check for the same.

In [283]:
df_1 = df.loc[:,['loan_amnt','funded_amnt','funded_amnt_inv']]
sns.pairplot(df_1)
plt.show()

So as expected the 3 columns seems to be linearly corelated from the pairplot.

In [284]:
df_1 = df.loc[:,['loan_amnt','funded_amnt','funded_amnt_inv',"installment"]]
sns.heatmap(df_1.corr(),annot = True)

Hence they show high correlation of > 0.9

Since these three seem to be  highly correlated, we can drop three of them.
Lets keep loan amount and drop other two.

In [285]:
list_redundant = ["funded_amnt", "funded_amnt_inv","installment"]
df = df.drop(list_redundant,axis =1)

From issue_d ie issued date, we can get the dereived metric "month".
The year information wont be useful for future prediction.
Hence we can get derieved metric "month" and then drop the original variable "issue_d"

In [287]:
df["month"] =  df.issue_d.apply(lambda x: x[:3])
df = df.drop(["issue_d"],axis =1)

# Null values imputation analysis

Lets check for number of null values in each column.

In [288]:
df.isna().sum()

There are some columns with some missing values:
emp_title,emp_length,title,mths_since_last_delinq,revol_util,last_pymnt_d,                   last_credit_pull_d,pub_rec_bankruptcies    


Now lets consider which metric can be used to impute null values.

Now lets see datatype of these columns.

In [289]:
df.columns

In [290]:
df_2 = df.loc[:,["emp_title","emp_length","title","mths_since_last_delinq","revol_util","last_pymnt_d","last_credit_pull_d","pub_rec_bankruptcies"]]
df_2.info()

emp_title,emp_length,title,revol_util,last_pymnt_d, last_credit_pull_d are categorical variables.
Hence they can be imputed with corresponding "mode value".

For numeric variables, lets see their spread

In [291]:
df_3 = df.loc[:,["mths_since_last_delinq"]]
df_3.boxplot()

Since it has outliers, hence the median metric should be used for imputation.

In [292]:
df.pub_rec_bankruptcies.value_counts()


Since it has only 3 values, it can be imputed using median metric ie most ocurring value.

Now check is any rows are missing high percentage of values.

Now lets see percentage of missing values for each row.
We will drop if percentage is > 80%.

In [293]:
#no rows dropped wrt missing values
df.isnull().sum(axis=1).value_counts()

Since row wise missing values are low, hence no rows can be dropped.

# DATA ANALYSIS

Now this dataset include current loans also ie ones where loans are going on.
Hence it include customer behaviour variable also ie information based on customer after his 
loan is approved like last payment date.
These information are not available at time of loan application.
So they are out of scope for this analysis and hence can be dropped.
These columns are chosen according to the domain knowledge.

Remove Consumer Behaviour variables as they are not available at time of loan application.

In [294]:
list_consumer_behaviour = ['delinq_2yrs', 'earliest_cr_line','out_prncp', 'out_prncp_inv','open_acc','revol_bal', 'revol_util','total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'delinq_amnt','last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d','mths_since_last_delinq','acc_now_delinq','recoveries',"inq_last_6mths","total_acc","collection_recovery_fee"]
df = df.drop(list_consumer_behaviour,axis =1)

Now title and purpose both give for what loan was applied.
Lets see their categories.

In [295]:
df.title.value_counts()

In [296]:
df.purpose.value_counts()

In [186]:
df.title.nunique()

We see that purpose column is better for analysis as title column has similar categories grouped as
different categories.
Example title has categories such Debt Consolidation,Debt Consolidation Loan, Consolidation,debt consolidation  
which all mean the same.
Hence we can use purpose column and drop title column.

In [297]:
df = df.drop(["title"],axis =1)

Now lets see shape of the data now.

In [298]:
df.shape

So there are 18 columns including the target variable.
Lets see information on these columns.

In [299]:
df.info()

Preparing target variable.
Now lets seevalue counts of target variable.

In [300]:
df.loan_status.value_counts()

Since the "current" loans are ongoing loans, they cant be used for prediction as we dont have their 
ground truth ie they default or not.
Hence these rows can be dropped

In [301]:
df = df[~(df.loan_status == "Current")]
df.shape

For analysis we have to convert target variable to numerical.
Hence "Charged off" will be imputed with 1 and "fully_paid" with 0.

In [302]:
df.loan_status = df.loan_status.apply(lambda x: 1 if x =="Charged Off" else 0 )

In [303]:
df.loan_status.value_counts()

In [304]:
sns.countplot(df.loan_status)
plt.show()

We see that there are > 5k datapoints are defaulted loans.

# 1. Univariate Analysis

Lets see the variables we will consider

In [305]:
df.info()

We will do outlier analysis along with univariate analysis.

i)loan amount

In [306]:
sns.distplot(df.loan_amnt)
plt.show()

##### We see that loan amount is skewed to lower side ie higher number of people are applying
for lower amount of loan and vice versa.
We also peaks at 5k, 10k, 15k etc. Hence more loans are given in multiples of 5k.

Now lets do outlier analysis for loan amount and see the numerical spread.

In [307]:
sns.boxplot(df.loan_amnt)
plt.show()

We see that there are outlier in the range 30k to 35k and majority of loans are between 5k and 15k.

ii)Term

In [308]:
sns.countplot(df.term)
plt.show()

We see that more people have applied for the shorter term of 36 months.
This is consistent with fact that more people applied for lower amounts of loan.

iii) Interest rate

In [309]:
sns.displot(df.int_rate)
plt.show()

We see that interest rate are more spread out.
There is downward trend of number of loans with increase of interest rate.
This is consistent with trend in loan amount as usually higher loan amounts attract higher interest rates.

In [310]:
sns.boxplot(df.int_rate)
plt.show()

We see ouliers are present in range above 22.5%.

iv) Employment Length

In [311]:
sns.countplot(df.emp_length)
plt.show()

We see that there is downward trend as employment length increase.
We also see high peaks for categories 1 and 10 years.
This could be due to fact that be binned all <1 years to 1 year and all >10 years to 10 years.

v)annual_income

In [313]:
sns.distplot(df.annual_inc)
plt.show()

We see that there are many outliers and majority are less than 50k income.

In [314]:
df.annual_inc.describe()

Minimum is 4000 while max is 60k.

In [315]:
sns.boxplot(df.annual_inc)
plt.show()

Outliers are present with high income like 40k and 60k.

vi) Grade

In [316]:
sns.countplot(df.grade.sort_values())
plt.show()

This shows that more loans are given in least risk grade A and increases as risk category
increases to G.

vii)Purpose

In [317]:
sns.countplot(df.purpose)
plt.xticks(rotation =90)
plt.show()

We see that more people apply for loans for purposes of debt consolidation and credit cards while less
people apply for renewable energy and education.

viii) Home ownership

In [318]:
sns.countplot(df.home_ownership)
plt.show()

We see that people with houses on rent or mortage apply for most loans while people with
own house apply for less loans.

ix) Employment title

In [319]:
df.emp_title.nunique()

In [320]:
df.emp_title.value_counts()

We see that there are over 28k employment titles and highest being US Army and Bank of America

x) Verification status

In [321]:
sns.countplot(df.verification_status)
plt.show()

We see most people applying for loan are not verified and least being source verified.

xi) Month of the year

In [322]:
sns.countplot(df.month)
plt.show()

We see that loan applications increase from January to December.

xii)number of public debt bankruptcies

In [323]:
sns.countplot(df.pub_rec_bankruptcies)
plt.show()

We see that most loans are given if person has no public bankruptcies
while few are given to people with one , it is rare for people with 2 public bankruptcies.

xiii) Zip code

In [324]:
sns.distplot(df.zip_code)
plt.show()

We see that people at higher and lower end of zipcode apply for more loans.

xiv) Address state

In [325]:
plt.figure(figsize=(10, 5))
sns.countplot(df.addr_state)
plt.xticks(rotation =90)
plt.show()

We see that people from states like CA and NY get more loans while those from ME, IN get least.

xv) Debt to income ratio(dti)

In [326]:
sns.distplot(df.dti)

We see that people with moderate dti get loans while those above 25% dti get less loans.

In [327]:
sns.boxplot(df.dti)
plt.show()

We see that no outliers are present for debt to income ratio.

xvi)Number of derogatory public records

In [328]:
sns.countplot(df.pub_rec)
plt.show()

We see that it gets difficult to get loan if there are derogatory public records for a person but 
few with one record also get loans.

# Outlier Removal Analysis

We see that the following columns have outliers.
- annual_inc, int_rate, loan_amnt.

They can be removed using inter quartile range(IQR) as follows.

Q3 = np.percentile(0.75)
Q1 = np.percentile(0.75)
IQR = Q3-Q1
lower_range = Q1 - 1.5*IQR
higher_range = Q3 +1.5*IQR

df_without_outliers = [(df["annual_inc"] < lower_range) | (df["annual_inc"] > upper_range)\



### Univariate with segmented

We will also be considering the information content of each column wrt default rate.
It will tell us how much difference in values it has between defaulted and nondefaulted loans.
The larger the difference in values , larger is its information content.

We will use a function to compute information content of each column and use it during univariate segment analysis.
It is tell which columns should get higher priority and most effect on loan status.

In [329]:
# Global dictionary that store information values
dict_info = {}
def get_info_content(plot):
    list_bar_values = []
    #Get height of each bar in barplots
    for i in plot.patches:
        list_bar_values.append(float('{:.2f}'.format(i.get_height())))
    #Return difference between highest and smallest bars in the plot
    info = max(list_bar_values) - min(list_bar_values)  
    return float('{:.2f}'.format(info))

For segemnted analysis, we will convert numeric cols to categorical using binning.

In [330]:
df['loan_amnt_bin'] = pd.cut(x=df['loan_amnt'], bins=[0,5000, 9600, 35000],
                    labels=['Low', 'Medium', 'High'])

df['annual_inc_bin'] = pd.cut(x=df['annual_inc'], bins=[0,40000, 50000, 6000000.0],
                    labels=['Low', 'Medium', 'High
                            
df['int_rate_bin'] = pd.cut(x=df['int_rate'], bins=[0,8, 12, 25],
                    labels=['Low', 'Medium', 'High
df['zip_code_bin'] = pd.cut(x=df['zip_code'], bins=[0,201, 452, 1000],
                    labels=['Low', 'Medium', 'High'])
df["dti_bin"] = pd.cut(x=df['dti'], bins=[0,10, 20, 30],
                    labels=['Low', 'Medium', 'High'])

Checking if all categorical are binned into numeric type.

In [385]:
df_1 = df.loc[:,["loan_amnt_bin","annual_inc_bin","int_rate_bin","zip_code_bin","dti_bin"]]
df_1.head()

i) Loan amount

In [331]:
p = sns.barplot(x = df.loan_amnt_bin,y=df.loan_status)
info = get_info_content(p)
dict_info.update({"Loan amount": info })

Insight:
Default rate is comparatively high for higher loan amounts

Recommendation:
When giving loans of higher amount, we should be cautious as default rate is more.



ii)Term


In [332]:
p = sns.barplot(x = df.term,y=df.loan_status)
info = get_info_content(p)
dict_info.update({"term": info })

Insight:
Default rate is comparatively high for higher term of 60 months.

Recommendation:
The term of 36 months should be given as default option to reduce loans in 60 months term.

iii) Interest Rate

In [334]:
p = sns.barplot(x = df.int_rate_bin,y=df.loan_status)
info = get_info_content(p)
dict_info.update({"Interest rate": info })

Insight:
Default rate rises as the interest rate rises.

Recommendation:
Giving lower rate of interest could decrease defaults but could also reduce profits from 
interest payments.
Hence medium interest rate should be promoted as a trade off.

iv) Grade

In [335]:
p = sns.barplot(df.grade.sort_values(),df.loan_status)
info = get_info_content(p)
dict_info.update({"Grade": info })

Insight:
Default rate increases as the risk grade increase.

Recommendation:
Loans of lower risk grade should be promoted and be cautius of higher risk grade loans.

v) Sub Grade

In [337]:
p = sns.barplot(df.sub_grade.sort_values(),df.loan_status)
info = get_info_content(p)
dict_info.update({"sub_grade": info })
plt.xticks(rotation =90)
plt.show()

Insight:
Default rate is increases as the subgrade increases within a grade.
The different pattern for grade G should be neglected as it has lower samples for analysis as
seen from its error rate.

Recommendation:
When giving loans, lower subgrade loans should be promoted within a grade.

vi) Employment length

In [338]:
p = sns.barplot(x = df.emp_length, y=df.loan_status)
info = get_info_content(p)
dict_info.update({"Employment length": info })

Insight:
There is no much difference in default rate across different employment lengths.

Recommendation:
None

vii) Annual income

In [339]:
p = sns.barplot(x = df.annual_inc_bin, y=df.loan_status)
info = get_info_content(p)
dict_info.update({"annual_inc": info })

Insight:
Default rate is higher among people of lower annual income.

Recommendation:
When giving loans to people of lower annual income, we should be cautious.

viii) Zip code

In [341]:
p = sns.barplot(x = df.zip_code_bin, y=df.loan_status)
info = get_info_content(p)
dict_info.update({"zip_code": info })

Insight:
Default rate doesnt show much difference across zip codes.

Recommendation:
None

ix) Purpose of loan

In [342]:
p = sns.barplot(x = df.purpose, y=df.loan_status)
info = get_info_content(p)
dict_info.update({"purpose": info })
plt.xticks(rotation = 90)
plt.show()

Insight:
Default rate is highest for small business loans and lowest for categories like cars.
This is a good indictor of default as default rate is almost double of other categories.

Recommendation:
If possible reduce small business loans and promote loans for credit card and car etc.

x) Verification status

In [344]:
p = sns.barplot(x = df.verification_status, y=df.loan_status)
info = get_info_content(p)
dict_info.update({"verification status": info })
plt.show()

Insight:
Default rate is lowest when not verified. This goes against popular belief.
But since the difference is rate is low(<0.05), verification status cannot be taken as driving factor of default.



xi) Month of the year

In [345]:
p = sns.barplot(x = df.month, y=df.loan_status)
info = get_info_content(p)
dict_info.update({"month": info })
plt.xticks(rotation = 90)
plt.show()

Insight:
Default rate is almost similar across months. Hence not driving factor of default.


xii) Address state

In [348]:
df.addr_state.value_counts()

Since no of states are high and some states have very low data samples,
we will use the top 20 states with highest no of samples for analysis.

In [346]:
#slicing out only data of top 20 states count wise
top_12_addr_state = (list(df.addr_state.value_counts().index))[:20]
df_4 = df[df.addr_state.isin(top_12_addr_state)]

In [347]:
plt.figure(figsize=(10, 5))
p = sns.barplot(x = df_4.addr_state, y=df_4.loan_status)
info = get_info_content(p)
dict_info.update({"addr_state": info })
plt.xticks(rotation =90)
plt.show()

Insight:
Default rate is similar across different states and hence not a driving factor.


xiii) Employment title

In [350]:
df.emp_title.value_counts()

Here also no of categories are huge ie >28k. Hence we will use top 30 categories by count.

In [351]:
# Slicing data of onlt top 30 categories of emp title
top_emp_title = (list(df.emp_title.value_counts().index))[:30]
df_4 = df[df.emp_title.isin(top_emp_title)]
plt.figure(figsize=(10, 5))

p = sns.barplot(x = df_4.emp_title, y=df_4.loan_status)
info = get_info_content(p)
dict_info.update({"emp_title": info })
plt.xticks(rotation =90)
plt.show()

Insight:
Though there is variation in defaul rate among emp titles, error line are quite high for all.
This is because most of have very few data points for analysis(<100 mostly).
Hence no correct insight can be made.


xiv) Home ownership

In [352]:
df.home_ownership.value_counts()

We see that OTHER and NONE have very few samples and will give higherror rate during analysis. Hence removing them for analysis.

In [353]:
#We can remove the non countable value compare to other
list_home_ownership = ["RENT","OWN","MORTGAGE"]
df_5 = df[df.home_ownership.isin(list_home_ownership)]

p = sns.barplot(x = df_5.home_ownership, y=df_5.loan_status)
info = get_info_content(p)
dict_info.update({"home_ownership": info })

Insight:
No considerable difference in default rate across categories.
Hence not a driving factor.

Recommendation:
None

xv) Debt to income ratio


In [363]:
p = sns.barplot(x = df.dti_bin, y=df.loan_status)
info = get_info_content(p)
dict_info.update({"dti": info })

Insight:
Default rate increase as dti increases

Recommendation:
When giving loans , prefer people with lower dti.

xvi) no of public derogatory records

In [367]:
df.pub_rec.value_counts()

Since samples for categories 2,3,4 are very low, we remove them for better analysis.

In [368]:
df_1 = df[df.pub_rec<=1]

In [369]:
p = sns.barplot(x = df_1.pub_rec, y=df_1.loan_status)
info = get_info_content(p)
dict_info.update({"pub_rec": info })

Insight:
Default rate is higher for more derogatory public records.

Recommendation:
Avoid giving loans to people with derogatory public records.

xvii) no of public bankruptcies

In [371]:
df.pub_rec_bankruptcies.value_counts()

Removing category 2 as it has very few samples for better analysis.

In [376]:
df_1 = df[df.pub_rec_bankruptcies<=1]
p = sns.barplot(x = df_1.pub_rec_bankruptcies, y=df_1.loan_status)
info = get_info_content(p)
dict_info.update({"pub_rec_bankruptcies": info })

Insight:
Default rate is higher among people with more public bankruptcies.

Recommendation:
Avoid giving loans to people of more public bankruptcies.

# Information Content Analysis

In [384]:
df_info = pd.DataFrame(list(dict_info.items()))
df_info.rename(columns = {0:"Feature",1: "Info content"},inplace = True)
df_info.sort_values("Info content",ascending = False).reset_index()

Insight:
Stronger driving factors of loan default are:
Grade, subgrade, purpose , term, pub_rec and pub_rec_bankruptcies.
Emp title is not considered due to low sample per categories.

Recommendation:
These factors should be given priority while sanctioning loans.

# 2. Bivariate Analysis

i) Interest rate and loan amount

In [244]:
sns.boxplot(df.int_rate, df.loan_amnt_bin)
plt.show()

From boxplot, we can see that higher the loan amount, higher is the interest charged,

Now lets see how they relate with default rate.

In [245]:
p = sns.barplot(x = df.int_rate_bin,y=df.loan_status, hue=df.loan_amnt_bin)
plt.show()

Insight:
We can see that as interest rate increases, the default rate also increases for all loan amounts.
Default rate is highest when high loan amounts are charged at higher interest rate.

Recommendation:
Hence to reduce default, medium interest rate can be charged for high loan amount,

ii) Loan amount and purpose of loan

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(df.purpose, df.loan_amnt)
plt.xticks(rotation =90)
plt.show()

We can see that higher loan amounts are given for small business while purposes like car and
vacation get lower loan amounts.

Now lets see how they relate with default rate.

In [ ]:
p = sns.barplot(x = df.loan_amnt_bin,y=df.loan_status, hue=df.purpose)
plt.legend(bbox_to_anchor =(1.2,0.8))
plt.xticks(rotation =90)
plt.show()

Insight:
We can see that default rate is higher than usual(close  to30%) when higher loan amt given for small business.

Recommendation:
Hence small business should be given only low or medium amount of loan to reduce defaults.

iii) interest rate and purpose

In [ ]:
sns.boxplot(df.int_rate, df.purpose)
plt.show()

We can see that higher interest rate is applied to categories like small business and house while lower interest rate is offered for loans for cars and vacation.

Now lets see how they relate with default rate.

In [ ]:
p = sns.barplot(x = df.int_rate_bin,y=df.loan_status, hue=df.purpose)
plt.legend(bbox_to_anchor =(1.05,0.9))
plt.ylabel("Default rate")
plt.show()

Insight:
Here we can see that small business loans have highest default rate among all categories 
  while default rate increases as interest rate increase across all categories of purpose.
So when small business loans are given at high interest rate, the default rate is highest.
Educational loans even at low interest rate show high default rate in the plot.
 But since it also has high error line due to lower data points, it is neglected in analysis.
 
Recommendation:
 So for higer profit from interest recepits, at trade off can be reached if for small business loans
 are given at lower interest rate while medium interest rate can be charged for other categories of purpose.

iv) Loan amount and grade

In [ ]:
sns.boxplot(df.grade.sort_values(), df.loan_amnt)

We can see that higher loan amounts are given when grade increases.

In [ ]:
p = sns.barplot(x = df.loan_amnt_bin,y=df.loan_status, hue=df.grade.sort_values())

Insight:
Default rate are almost similar when we take into account error rate also.
Hence no insight can be inferred from this.

v) Term and annual income

In [ ]:
sns.boxplot(df.term, df.annual_inc)
plt.show()

There are outliers in annual income. 

In [ ]:
df.annual_inc.describe(percentiles = [0.95])

So 95 percenatage of data points of annual income falls under 1.5 lakh.
hence removing outliers.

In [ ]:
df_2 = df[df.annual_inc <= 150000]
sns.boxplot(df_2.term, df_2.annual_inc)
plt.show()

We can see that people with higher annual income are offered loan at longer term ie 60 months.
Now lets see how they relate with loan default rate.

In [ ]:
sns.barplot(x = df.term,y=df.loan_status, hue=df.annual_inc_bin)

Insight:
We see that default rate is high(>30%)when people with low annual income are given higher term.

Recommendation":
Hence this should be avoided and they should be given shorter term of 36 months.

vi) Term and Purpose

In [ ]:
sns.countplot(x='term',hue='purpose',data=df)
plt.legend(bbox_to_anchor = (1.05,0.8))
plt.show()

We see that more people have applied for shorter term especially for credit card and small business.

In [ ]:
p = sns.barplot(x = df.term,y=df.loan_status, hue=df.purpose)
plt.legend(bbox_to_anchor = (1.05,0.9))
plt.show()

Insight:
We see that high default rate for small business loans offered at longer term.

Recommendation:
Hence to reduce defaults, small business loans should be given at lower term of 36 months.

vii) Loan amount and term

In [ ]:
sns.boxplot(df.term, df.loan_amnt)
plt.show()

We see that higher loan amounts attracts higher term.
Lets see how they relate with default rate.

In [ ]:
sns.barplot(x = df.loan_amnt_bin,y=df.loan_status, hue=df.term)
plt.show()

Insight:
No insights can be inferred as default rate is similar across all categories of loan amount
wrt term

viii) Term and interest rate

In [ ]:
sns.boxplot(df.term, df.int_rate)
plt.show()

We see that higher interest rate is given when term is longer.
Now lets see how they relate with default rate.

In [ ]:
sns.barplot(x = df.int_rate_bin,y=df.loan_status, hue=df.term)
plt.show()

Insight:
We see that default rate increase across both terms as the interest rate increases.

recommendation:
Hence for higher term, low or medium interest rate should be given to reduce defaults.

ix) Interest rate and grade

In [ ]:
sns.boxplot(df.int_rate, df.grade.sort_values())
plt.show()

We see that higher interest rate are applied as the grade increases.
This is consistent with the fact that risk of loan increases as grade moves from A to G.

In [ ]:
sns.barplot(x = df.int_rate_bin,y=df.loan_status, hue=df.grade.sort_values())
plt.show()

Insight:
No insight can be inferred from the plot.

x) Annual income and loan amount

In [ ]:
sns.boxplot(df.annual_inc_bin, df.loan_amnt)
plt.show()

We see that people with higher annual income aplly for higher amount of loans.
Now lets see how they relate with default rate.

In [ ]:
sns.barplot(x = df.annual_inc_bin,y=df.loan_status, hue=df.loan_amnt_bin)
plt.show()

Insight:
We see that people with low annual income default more amomg all categories of loan amount.
Low income with high loan amount lead to default rate >20%.

Recommendation:
Hence people with low annual income should be given onlu low or medium amount of loan.

xi) Annual income and interest rate

In [ ]:
sns.boxplot(df.int_rate, df.annual_inc_bin)
plt.show()

We see that interest rate doesnt depend soley on annual income.

In [ ]:
sns.barplot(x = df.annual_inc_bin,y=df.loan_status, hue=df.int_rate_bin)
plt.show()

Insight:
We see that low annual income at high interest rate lead to higher default.
Recommendation:
Hence people with low income should only be charged low or medium interest rate.

xii) Annual income purpose

In [ ]:
sns.boxplot(df_2.annual_inc, df_2.purpose)
plt.show()

We see that people with higher income take loans for purposes like home improvement while
those with low income take for purposes like education.

In [ ]:
p = sns.barplot(x = df.annual_inc_bin,y=df.loan_status, hue=df.purpose)
plt.legend(bbox_to_anchor = (1.05,0.9))
plt.show()

Insight:
We see that default rate is high(> 30%) when low income people are given loan for small business.

Recommendation:
Hence this combination should be avoided and small business loans should be given to people with 
medium or high income.

xiii) Loan amount and no of public bankruptcies

In [ ]:
df.pub_rec_bankruptcies.value_counts()

In [ ]:
sns.boxplot( df.pub_rec_bankruptcies,df.loan_amnt,)

We see that higher loan amounts are given for people with lower public bankruptcies.

In [ ]:
sns.barplot(x = df.pub_rec_bankruptcies,y=df.loan_status, hue=df.loan_amnt_bin)

Insight:
Default rate is high when people with higher public bankruptcies are given higher loan amounts.
Recommendation:
So to reduce default rate, people with higher public bankruptcies should be given only low loan amounts.